In [ ]:
from __init__ import *

In [ ]:
signal_table_ = []

for data_index, data in enumerate(SETTING["model_data_"]):

    signal_table = pd.read_csv(
        "../output/signal/{}.tsv".format(data_index), sep="\t", index_col=0
    )

    signal_table.columns.name = data["axis_1_name"]

    signal_table_.append(signal_table)

In [ ]:
if SETTING["model_mode"] == "deep":

    matrix_factorization_mode = "ws"

    sync_axis = 1

elif SETTING["model_mode"] == "range":

    matrix_factorization_mode = "hs"

    sync_axis = 0

signal_table_ = kraft.table.sync_axis(signal_table_, sync_axis)

In [ ]:
for r in SETTING["resolution_"]:

    untangle_ = np.asarray(
        tuple(kraft.table.untangle(signal_table) for signal_table in signal_table_),
        dtype="object",
    )

    w_, h_, error__ = kraft.matrix_factorization.factorize(
        untangle_[:, 0], matrix_factorization_mode, r
    )

    directory_path = "../output/matrix_factorization_r{}/".format(r)

    kraft.path.path(directory_path)

    kraft.matrix_factorization.plot_factorization(
        w_,
        h_,
        untangle_[:, 1],
        untangle_[:, 2],
        untangle_[:, 3],
        untangle_[:, 4],
        error__,
        directory_path=directory_path,
    )

    factor_label_ = kraft.matrix_factorization.make_factor_label_(r)

    w_table_ = tuple(
        kraft.table.entangle(
            w, untangle_[w_index, 1], factor_label_, untangle_[w_index, 3], "Factor"
        )
        for w_index, w in enumerate(w_)
    )

    h_table_ = tuple(
        kraft.table.entangle(
            h, factor_label_, untangle_[h_index, 2], "Factor", untangle_[h_index, 4]
        )
        for h_index, h in enumerate(h_)
    )

    file_path_template = "{}{{}}_{{}}.tsv".format(directory_path)

    for w_index, w_table in enumerate(w_table_):

        w_table.to_csv(file_path_template.format("w", w_index), sep="\t")

    for h_index, h_table in enumerate(h_table_):

        h_table.to_csv(file_path_template.format("h", h_index), sep="\t")